In [2]:
from lmfit import minimize, Parameters
import pandas as pd
import tellurium as te
import numpy as np

1. define the equation
2. fill out all the matrices
    - import the data
3. use lmfit to  fit the equation


In [3]:
# import the data 
df = pd.read_csv('data/Simplified_Teusink_yeast_1.5.csv')

In [4]:
r = te.loada('Simplified_Teusink_yeast.ant')
N=r.getFullStoichiometryMatrix()

In [5]:
enzymes = ['e_' + i for i in r.getReactionIds()]
internal = r.getFloatingSpeciesIds()
external = r.getBoundarySpeciesIds()
fluxes = ['v_' + i for i in r.getReactionIds()]

In [6]:
v_star = df[fluxes].loc[0]

In [7]:
en = df[enzymes].loc[1]/df[enzymes].loc[0]
xn = df[internal].loc[1]/df[internal].loc[0]
yn = df[external].loc[1]/df[external].loc[0]
vn = df[fluxes].loc[1]/df[fluxes].loc[0]

In [8]:
ex = r.getScaledElasticityMatrix()

In [9]:
a = r.getBoundarySpeciesIds()
b = r.getReactionIds()
c=[]
for i in b: 
    for ii in a: 
        c.append(r.getUnscaledParameterElasticity(i, ii) * r[ii]/r[i])
ey = np.array(c).reshape((len(b),len(a)))

### testing with one perturbation

In [10]:
# perturbed_fluxes = np.diag(v_star)@(np.ones(N.shape[1]).reshape((-1,1)) + ex@np.log(xn).T + ey@np.log(yn).T)
b = np.ones(N.shape[1]).reshape((-1,1)) + np.expand_dims(ex@np.log(xn).T, axis=1) + np.expand_dims(ey@np.log(yn).T, axis=1)
(np.diag(v_star) @ b) - np.expand_dims(df[fluxes].loc[1].values, axis=1)

array([[-1.04308180e-01],
       [-2.46068925e-02],
       [-4.40773506e-04],
       [-2.25284438e-03],
       [-3.16194718e-02],
       [-2.73733356e-02],
       [ 6.36301101e-03],
       [ 2.74993145e-03],
       [ 4.97888552e-04],
       [ 4.32233945e-02],
       [-3.01730752e-02],
       [-2.34042274e-02],
       [-9.82060973e-05],
       [-1.48933918e-01],
       [-1.10072363e-03],
       [-1.28018080e-03]])

### testing with multiple perturbations

In [11]:
en = df[enzymes].loc[1:]/df[enzymes].loc[0]
xn = df[internal].loc[1:]/df[internal].loc[0]
yn = df[external].loc[1:]/df[external].loc[0]
vn = df[fluxes].loc[1:]/df[fluxes].loc[0]

In [12]:
b = np.ones((16,19)) + ex@np.log(xn).T + ey@np.log(yn).T
(np.diag(v_star) @ b).T - df[fluxes].loc[1:].values


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,-0.104308,-0.024607,-4.407735e-04,-2.252844e-03,-0.031619,-0.027373,0.006363,0.002750,0.000498,0.043223,-0.030173,-0.023404,-0.000098,-0.148934,-0.001101,-0.001280
2,-0.057189,-0.161700,-8.271669e-05,-4.227728e-04,-0.078654,-0.066613,0.016331,0.007040,0.001795,0.102640,-0.071643,-0.055434,-0.000541,-0.041561,-0.003180,-0.007600
3,-0.028914,0.005550,-5.317629e-02,-1.085102e-04,0.006963,0.006145,-0.001355,-0.000588,-0.000063,-0.009904,0.006909,0.005371,-0.000005,-0.021022,0.000199,-0.000064
4,-0.259434,0.050952,-1.654949e-03,-2.399490e-01,0.061738,0.056079,-0.011296,-0.004952,0.000112,-0.093114,0.064805,0.050538,-0.000480,-0.187852,0.001027,-0.005295
5,-0.039255,-0.045725,-3.907242e-05,-1.997047e-04,0.194592,-0.045901,0.010953,0.004727,0.001028,0.071650,-0.050019,-0.038753,-0.000267,-0.028536,-0.002018,-0.003605
6,-0.036466,-0.042536,-3.373207e-05,-1.724088e-04,-0.156913,-0.129886,0.010136,0.004375,0.000925,0.066736,-0.046589,-0.036102,-0.000232,-0.026510,-0.001849,-0.003114
7,-0.017478,-0.020577,-7.770472e-06,-3.971419e-05,-0.073701,-0.021345,-0.197834,0.008249,0.002250,0.118004,-0.082354,-0.063681,-0.000711,-0.012710,-0.003816,-0.004268
8,-0.013024,-0.015366,-4.317651e-06,-2.206921e-05,-0.054677,-0.016049,-0.147205,0.005939,0.001412,0.088163,-0.061544,-0.047649,-0.000401,-0.009472,-0.002617,-0.002354
9,-0.008949,-0.010578,-2.039500e-06,-1.042211e-05,-0.037417,-0.011113,-0.101020,-0.228499,-0.287121,0.060726,-0.042393,-0.032860,-0.000193,-0.006508,-0.001648,-0.001105
10,-0.005367,-0.006355,-7.335886e-07,-3.750452e-06,-0.022365,-0.006709,-0.060534,-0.134009,-0.104006,-0.529793,-0.025483,-0.019772,-0.000070,-0.003904,-0.000906,-0.000396


### using lmfit to predict elasticity values 

In [20]:
16 * 11

176

In [48]:
# ask Herb about uncertainty
# https://lmfit.github.io/lmfit-py/intro.html

def residual(params, xn, yn, df):
    
    some_list = []
    Ex_params = [i for i in params if 'Ex' in i]
    for i in Ex_params:
        some_list.append(params[i])
    ex = np.asarray(some_list).reshape((16,11))
    some_list = []
    Ey_params = [i for i in params if 'Ey' in i]
    for i in Ey_params:
        some_list.append(params[i])
    ey = np.asarray(some_list).reshape((16,3)) 

    model = np.ones((16,19)) + ex@np.log(xn).T + ey@np.log(yn).T

    return (df[fluxes].loc[1:].values-model.T)

params = Parameters()

for i in range(16*(11)):
    params.add('Ex'+str(i), value=0, min=-10, max=10)
for i in range(16*(3)):
    params.add('Ey'+str(i), value=0, min=-10, max=10)

out = minimize(residual, params, args=(xn, yn, df), method='differential_evolution')